In [8]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

In [9]:
clmns = [
    "Open 1","Close 1","Open 2","Close 2","Open 3","Close 3","Open 4","Close 4","Open 5","Close 5","Open 6","Close 6","Open 7","Close 7","Open 8","Close 8","Open 9","Close 9","Open 10","Close 10","Open 11","Close 11","Open 12","Close 12","Open 13","Close 13","Open 14","Close 14","Open 15","Close 15","Open 16","Close 16","Open 17","Close 17","Open 18","Close 18","Open 19","Close 19","Open 20","Close 20","Open 21","Close 21","Open 22","Close 22","Open 23","Close 23","Open 24","Close 24","Open 25","Close 25","Open 26","Close 26","Open 27","Close 27","Open 28","Close 28","Open 29","Close 29","Open 30","Close 30",
    "Open 31","Close 31","Open 32","Close 32","Open 33","Close 33","Open 34","Close 34","Open 35","Close 35","Open 36","Close 36","Open 37","Close 37","Open 38","Close 38","Open 39","Close 39","Open 40","Close 40","Open 41","Close 41","Open 42","Close 42","Open 43","Close 43","Open 44","Close 44","Open 45","Close 45","Open 46","Close 46","Open 47","Close 47","Open 48","Close 48","Open 49","Close 49","Open 50","Close 50","Open 51","Close 51","Open 52","Close 52","Open 53","Close 53","Open 54","Close 54","Open 55","Close 55","Open 56","Close 56","Open 57","Close 57","Open 58","Close 58","Open 59","Close 59","Open 60","Close 60",
    "suggestion"]
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["buy","sell"], axis=1)
  y = data[["buy","sell"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-1,1))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(15, data.shape[0]):
        grow = []
        ggrow = []
        gggrow = []
        for x in range(1, 61):
            grow.append([data[i-x, 0], data[i-x, 1],
                        data[i-x, 2], data[i-x, 3]])
        for eeach in grow:
            ggrow.append(scaler(np.array(eeach).reshape(-1, 1)))
        for eachh in ggrow:
            eachh = np.array(eachh).reshape(-1,1)
            eachh = np.delete(eachh,1)
            eachh = np.delete(eachh, 1)
            gggrow.append(eachh)
        sugg = "sell"
        if data[i][3] > data[i][0]:
            sugg = "buy"
        arr = np.array(gggrow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)
    grow = []
    ggrow = []
    gggrow = []
    arr = []
    return np.array(row)

In [12]:
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 60:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        try:
           unattached_dfs.append(process(pd.read_csv(address)))
        except:
          print(" EP ")
     if np.array(unattached_dfs[0]).shape[0] == 0:
            print(" Null ")
            data = np.array(unattached_dfs[1])
            for z in unattached_dfs[2:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     else:
            data = np.array(unattached_dfs[0])
            for z in unattached_dfs[1:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     unattached_dfs = []
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")      

In [13]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"max","1mo")

Symbols : 1500
Data Folder Removed
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [14]:
folder_name = extract_data(10)
to_par(folder_name,1)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Files In Data : 38
Processing File:
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38  
(2757, 122)
(2205, 120) (2205, 2)
(552, 120) (552, 2)


In [15]:
model = Sequential()

model.add(Dense(1024, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))


opt = tf.keras.optimizers.Adamax()
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              123904    
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_2 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_3 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 2,225,154
Trainable params: 2,225,154
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(xTrain,yTrain,epochs=30,batch_size=8,validation_data=(xTest,yTest))

Epoch 1/30
276/276 [==============================] - 5s 17ms/step - loss: 0.5088 - accuracy: 0.7551 - val_loss: 0.3644 - val_accuracy: 0.8333
Epoch 2/30
276/276 [==============================] - 5s 17ms/step - loss: 0.2183 - accuracy: 0.9170 - val_loss: 0.3298 - val_accuracy: 0.8678
Epoch 3/30
276/276 [==============================] - 5s 17ms/step - loss: 0.0873 - accuracy: 0.9719 - val_loss: 0.4598 - val_accuracy: 0.8460
Epoch 4/30
276/276 [==============================] - 5s 17ms/step - loss: 0.0486 - accuracy: 0.9850 - val_loss: 0.4134 - val_accuracy: 0.8696
Epoch 5/30
276/276 [==============================] - 5s 17ms/step - loss: 0.0310 - accuracy: 0.9900 - val_loss: 0.4240 - val_accuracy: 0.8623
Epoch 6/30
276/276 [==============================] - 5s 17ms/step - loss: 0.0214 - accuracy: 0.9918 - val_loss: 0.4503 - val_accuracy: 0.8714
Epoch 7/30
276/276 [==============================] - 5s 17ms/step - loss: 0.0226 - accuracy: 0.9923 - val_loss: 0.4826 - val_accuracy: 0.8659

In [18]:
model.save("CMG10S_87.h5")

In [19]:
yf.download("btc-usd",period="max",interval="1mo")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-01,465.864014,468.174011,372.239990,386.944000,386.944000,410881000
2014-10-01,387.427002,411.697998,289.295990,338.321014,338.321014,902994450
2014-11-01,338.649994,457.092987,320.626007,378.046997,378.046997,659733360
2014-12-01,378.248993,384.037994,304.231995,320.192993,320.192993,553102310
2015-01-01,320.434998,320.434998,171.509995,217.464005,217.464005,1098811912
...,...,...,...,...,...,...
2022-01-01,46311.746094,47881.406250,33184.058594,38483.125000,38483.125000,923979037681
2022-02-01,38481.765625,45661.171875,34459.218750,43193.234375,43193.234375,671335993325
2022-03-01,43194.503906,48086.835938,37260.203125,45538.675781,45538.675781,830943838435


In [60]:
def process_for_prediction(data):
        i = -1
        if len(pd.DataFrame(data).columns) == 7:
          data = data.iloc[: , 1:]        
        data = np.array(data)
        grow = []
        ggrow = []
        gggrow = []
        for x in range(1, 61):
            grow.append([data[i-x, 0], data[i-x, 1],
                        data[i-x, 2], data[i-x, 3]])
        for eeach in grow:
            ggrow.append(scaler(np.array(eeach).reshape(-1, 1)))
        for eachh in ggrow:
            eachh = np.array(eachh).reshape(-1,1)
            eachh = np.delete(eachh,1)
            eachh = np.delete(eachh, 1)
            gggrow.append(eachh)
        arr = np.array(gggrow).flatten().reshape(-1,1)
        return arr
def make_prediction(symbol,period,timeframe):
    raw_data = process_for_prediction(yf.download(symbol,period=period,interval=timeframe))
    return  model.predict(np.array(raw_data).reshape(1,-1))

In [61]:
make_prediction("btc-usd","max","1mo")

[*********************100%***********************]  1 of 1 completed


array([[9.999944e-01, 9.427457e-06]], dtype=float32)

In [64]:
def process_for_prediction(data):
        i = -1
        row = []
        data.drop("symbol",axis=1,inplace=True)   
        data = np.array(data)
        grow = []
        ggrow = []
        gggrow = []
        for x in range(1, 61):
            grow.append([data[i-x, 0], data[i-x, 1],
                        data[i-x, 2], data[i-x, 3]])
        for eeach in grow:
            ggrow.append(scaler(np.array(eeach).reshape(-1, 1)))
        for eachh in ggrow:
            eachh = np.array(eachh).reshape(-1,1)
            eachh = np.delete(eachh,1)
            eachh = np.delete(eachh, 1)
            gggrow.append(eachh)
        arr = np.array(gggrow).flatten()
        return arr
def make_prediction(symbol,timeframe):
   tv = TvDatafeed()
   data = tv.get_hist(symbol='btcUSD',exchange='bitstamp',interval=Interval.in_monthly,n_bars=100)
   return model.predict(np.array(process_for_prediction(data)).reshape(1,-1))

In [65]:
make_prediction("s","x")

array([[9.9998701e-01, 2.0962376e-05]], dtype=float32)